# Precautions

### Here we discuss some precautions when using mdapy.

In [1]:
import mdapy as mp
import polars as pl
mp.init()

[Taichi] version 1.7.0, llvm 15.0.1, commit 2fd24490, win, python 3.8.18
[Taichi] Starting on arch=x64


In [2]:
mp.__version__

'0.10.0'

In [3]:
system = mp.System('../../../example/solidliquid.data')

In [4]:
print(system)

Filename: ../../../example/solidliquid.data
Atom Number: 8192
Simulation Box:
[[52.01907282  0.          0.        ]
 [ 0.         52.01907282  0.        ]
 [ 0.          0.         52.01907282]
 [-0.39698541 -0.39698541 -0.39698541]]
TimeStep: 0
Boundary: [1, 1, 1]
Particle Information:
shape: (5, 8)
┌──────┬──────┬───────────┬───────────┬───────────┬───────────┬──────────┬──────────┐
│ id   ┆ type ┆ x         ┆ y         ┆ z         ┆ vx        ┆ vy       ┆ vz       │
│ ---  ┆ ---  ┆ ---       ┆ ---       ┆ ---       ┆ ---       ┆ ---      ┆ ---      │
│ i64  ┆ i64  ┆ f64       ┆ f64       ┆ f64       ┆ f64       ┆ f64      ┆ f64      │
╞══════╪══════╪═══════════╪═══════════╪═══════════╪═══════════╪══════════╪══════════╡
│ 7913 ┆ 1    ┆ 2.64108   ┆ 3.2152    ┆ 1.74197   ┆ 2.90096   ┆ 1.72875  ┆ -1.77975 │
│ 4098 ┆ 1    ┆ -0.091857 ┆ 0.4658    ┆ 0.936675  ┆ -0.166803 ┆ 0.959803 ┆ 3.70312  │
│ 4414 ┆ 1    ┆ 51.3076   ┆ 2.93742   ┆ 2.12484   ┆ 4.50605   ┆ -1.06924 ┆ -1.41931 │
│ 4    ┆ 

### Check the position information

In [5]:
system.pos[:5]

array([[ 2.64108e+00,  3.21520e+00,  1.74197e+00],
       [-9.18574e-02,  4.65800e-01,  9.36675e-01],
       [ 5.13076e+01,  2.93742e+00,  2.12484e+00],
       [ 3.57773e+00, -2.44078e-02,  2.31072e+00],
       [ 4.52190e+01,  1.95654e+01, -2.41386e-01]])

Generally speaking, we do not need to change the position. So the pos information is inmutable for users.
If you try to change it, it will raise an error. This is for data safety.

In [6]:
system.pos[0, 0] = 1.

ValueError: assignment destination is read-only

If you really need to change ths positions for system object, you can modify the system.data and update it mannually!!!
Considering you want let atoms with z < 10 moves right 2 along x, you can do like below:

- The first way:

In [7]:
system.update_data(
    system.data.
    with_columns(
    pl.when(pl.col('z')<10).
    then(pl.col('x')+2).
    otherwise(pl.col('x')).
    alias('x')), 
    update_pos=True) 
# The first parameter is a new Dataframe, the second indicates mdapy will update the position information.

In [8]:
system.data.head() # x[0] from 2.64108 to 4.64108

id,type,x,y,z,vx,vy,vz
i64,i64,f64,f64,f64,f64,f64,f64
7913,1,4.64108,3.2152,1.74197,2.90096,1.72875,-1.77975
4098,1,1.9081426,0.4658,0.936675,-0.166803,0.959803,3.70312
4414,1,53.3076,2.93742,2.12484,4.50605,-1.06924,-1.41931
4,1,5.57773,-0.024408,2.31072,-6.04709,5.23451,9.18538
37,1,47.219,19.5654,-0.241386,-5.20306,3.79694,-2.15249


In [9]:
system.pos[:5] # updated correspondingly.

array([[ 4.6410800e+00,  3.2152000e+00,  1.7419700e+00],
       [ 1.9081426e+00,  4.6580000e-01,  9.3667500e-01],
       [ 5.3307600e+01,  2.9374200e+00,  2.1248400e+00],
       [ 5.5777300e+00, -2.4407800e-02,  2.3107200e+00],
       [ 4.7219000e+01,  1.9565400e+01, -2.4138600e-01]])

- The second way

In [10]:
system.data.replace(
    'y', 
    system.data.select(
    pl.when(pl.col('z')<10).
    then(pl.col('y')+2).
    otherwise(pl.col('y'))
    ).to_series()
) # Replace the 'y' directly.
system.update_pos() # update the position mannully.

In [11]:
system.data.head() # y[0] from 3.2152 to 5.2152

id,type,x,y,z,vx,vy,vz
i64,i64,f64,f64,f64,f64,f64,f64
7913,1,4.64108,5.2152,1.74197,2.90096,1.72875,-1.77975
4098,1,1.9081426,2.4658,0.936675,-0.166803,0.959803,3.70312
4414,1,53.3076,4.93742,2.12484,4.50605,-1.06924,-1.41931
4,1,5.57773,1.9755922,2.31072,-6.04709,5.23451,9.18538
37,1,47.219,21.5654,-0.241386,-5.20306,3.79694,-2.15249


In [12]:
system.pos[:5] # updated correspondingly.

array([[ 4.64108  ,  5.2152   ,  1.74197  ],
       [ 1.9081426,  2.4658   ,  0.936675 ],
       [53.3076   ,  4.93742  ,  2.12484  ],
       [ 5.57773  ,  1.9755922,  2.31072  ],
       [47.219    , 21.5654   , -0.241386 ]])

velocity is all the same with position.

In [13]:
system.vel[:5]

array([[ 2.90096 ,  1.72875 , -1.77975 ],
       [-0.166803,  0.959803,  3.70312 ],
       [ 4.50605 , -1.06924 , -1.41931 ],
       [-6.04709 ,  5.23451 ,  9.18538 ],
       [-5.20306 ,  3.79694 , -2.15249 ]])

If you try to modify it directly, raise an error.

In [14]:
system.vel[0, 0] = 3.5

ValueError: assignment destination is read-only

You can change the data to modify the velocities.

In [15]:
system.data[0, 'vx'] = 3.5

In [16]:
system.update_vel() # Update the velocity mannually.

system.data.head()

In [17]:
system.data.head() # vx[0] from 2.90096 to 3.5

id,type,x,y,z,vx,vy,vz
i64,i64,f64,f64,f64,f64,f64,f64
7913,1,4.64108,5.2152,1.74197,3.5,1.72875,-1.77975
4098,1,1.9081426,2.4658,0.936675,-0.166803,0.959803,3.70312
4414,1,53.3076,4.93742,2.12484,4.50605,-1.06924,-1.41931
4,1,5.57773,1.9755922,2.31072,-6.04709,5.23451,9.18538
37,1,47.219,21.5654,-0.241386,-5.20306,3.79694,-2.15249


In [18]:
system.vel[:5] # updated correspondingly.

array([[ 3.5     ,  1.72875 , -1.77975 ],
       [-0.166803,  0.959803,  3.70312 ],
       [ 4.50605 , -1.06924 , -1.41931 ],
       [-6.04709 ,  5.23451 ,  9.18538 ],
       [-5.20306 ,  3.79694 , -2.15249 ]])